In [1]:
import g2o
import numpy as np
from tqdm import tqdm

class GraphSLAM2D:
    def __init__(self, verbose=False) -> None:
        self.verbose = verbose
        self.optimizer = g2o.SparseOptimizer()
        self.solver = self._make_solver()          # ✅ 换成自动选择稀疏 solver
        self.algorithm = g2o.OptimizationAlgorithmLevenberg(self.solver)
        self.optimizer.set_algorithm(self.algorithm)

        self.vertex_count = 0
        self.edge_count = 0

    # ---------- 新增：根据可用类型自动选 solver ----------
    def _make_solver(self):
        """
        优先用稀疏：
          1) LinearSolverCSparseX
          2) LinearSolverCholmodX
          3) 最后兜底：LinearSolverDenseX
        """
        # 依次尝试这些名字
        candidates = [
            "LinearSolverCSparseX",
            "LinearSolverCholmodX",
            "LinearSolverDenseX",
        ]

        for name in candidates:
            if hasattr(g2o, name):
                LinearSolverCls = getattr(g2o, name)
                if self.verbose:
                    print(f"[GraphSLAM2D] Using solver: {name} + BlockSolverX")
                return g2o.BlockSolverX(LinearSolverCls())

        # 理论上不会走到这里，除非绑定太残
        raise RuntimeError("No suitable LinearSolverX found in g2o bindings!")

    # ------------------------------------------------------

    def vertex_pose(self, id):
        return self.optimizer.vertex(id).estimate()

    def vertex(self, id):
        return self.optimizer.vertex(id)

    def edge(self, id):
        return self.optimizer.edge(id)

    def add_fixed_pose(self, pose, vertex_id=None):
        v_se2 = g2o.VertexSE2()
        if vertex_id is None:
            vertex_id = self.vertex_count
        v_se2.set_id(vertex_id)
        if self.verbose:
            print("Adding fixed pose vertex with ID", vertex_id)
        v_se2.set_estimate(pose)
        v_se2.set_fixed(True)
        self.optimizer.add_vertex(v_se2)
        self.vertex_count = max(self.vertex_count, vertex_id + 1)

    def optimize(self, iterations=10, verbose=None):
        self.optimizer.initialize_optimization()
        if verbose is None:
            verbose = self.verbose
        self.optimizer.set_verbose(verbose)
        self.optimizer.optimize(iterations)
        return self.optimizer.chi2()


def load_posegraph_g2o(graph_slam: GraphSLAM2D, filename: str, fix_first: bool = True):
    opt = graph_slam.optimizer
    num_vertices = 0
    num_edges = 0

    with open(filename, "r") as f:
        for line in f:
            if not line.strip() or line.startswith("#"):
                continue

            parts = line.split()
            tag = parts[0]

            if tag == "VERTEX_SE2":
                _, vid, x, y, th = parts
                vid = int(vid); x = float(x); y = float(y); th = float(th)

                v = g2o.VertexSE2()
                v.set_id(vid)
                v.set_estimate(g2o.SE2(x, y, th))

                if fix_first and vid == 0:
                    v.set_fixed(True)

                opt.add_vertex(v)
                num_vertices += 1
                graph_slam.vertex_count = max(graph_slam.vertex_count, vid + 1)

            elif tag == "EDGE_SE2":
                (
                    _,
                    i, j,
                    dx, dy, dth,
                    I11, I12, I13, I22, I23, I33,
                ) = parts

                i = int(i); j = int(j)
                dx = float(dx); dy = float(dy); dth = float(dth)
                I11 = float(I11); I12 = float(I12); I13 = float(I13)
                I22 = float(I22); I23 = float(I23); I33 = float(I33)

                info = np.zeros((3, 3))
                info[0, 0] = I11
                info[0, 1] = info[1, 0] = I12
                info[0, 2] = info[2, 0] = I13
                info[1, 1] = I22
                info[1, 2] = info[2, 1] = I23
                info[2, 2] = I33

                e = g2o.EdgeSE2()
                e.set_vertex(0, opt.vertex(i))
                e.set_vertex(1, opt.vertex(j))
                e.set_measurement(g2o.SE2(dx, dy, dth))
                e.set_information(info)
                opt.add_edge(e)

                num_edges += 1
                graph_slam.edge_count += 1

    if graph_slam.verbose:
        print(f"Loaded {num_vertices} vertices, {num_edges} edges from {filename}")


In [2]:
# 1) 实例化
graph_slam = GraphSLAM2D(verbose=True)

# 2) 读 Intel g2o 文件（把路径改成你本地的）
g2o_file = "/home/yuzhou/Desktop/input_INTEL_g2o.g2o"  # 或你自己的路径
load_posegraph_g2o(graph_slam, g2o_file, fix_first=True)


opt = graph_slam.optimizer

[GraphSLAM2D] Using solver: LinearSolverCSparseX + BlockSolverX
Loaded 1228 vertices, 1483 edges from /home/yuzhou/Desktop/input_INTEL_g2o.g2o


In [ ]:
# 3) 保存一份优化前的位姿（就是 VERTEX_SE2 里的值）
vertices = opt.vertices()
pose_ids = sorted(
    vid for vid, v in vertices.items()
    if isinstance(v, g2o.VertexSE2)
)
poses_before = {
    vid: vertices[vid].estimate()
    for vid in pose_ids
}

print("=== Before optimization ===")
print("chi2 =", opt.chi2())
print("num poses:", len(pose_ids), "num edges:", len(opt.edges()))

print("First 5 poses (file init):")
for vid in pose_ids[-5:]:
    est = poses_before[vid]
    print(
        f"id={vid:3d}: "
        f"x={est.translation()[0]:.6f}, "
        f"y={est.translation()[1]:.6f}, "
        f"theta={est.rotation().angle():.6f}"
    )

# 4) 以这些 VERTEX_SE2 为初始值跑 LM
chi2_after = graph_slam.optimize(iterations=10000, verbose=True)

print("\n=== After optimization ===")
print("chi2 =", chi2_after)

#print("First 5 poses (after LM):")
for vid in pose_ids[-5:]:
    est0 = poses_before[vid]
    est1 = opt.vertex(vid).estimate()
    print(
        f"id={vid:3d}: "
        f"before=({est0.translation()[0]:.6f},{est0.translation()[1]:.6f},{est0.rotation().angle():.6f}), "
        f"after =({est1.translation()[0]:.6f},{est1.translation()[1]:.6f},{est1.rotation().angle():.6f})"
    )

=== Before optimization ===
chi2 = 0.0
num poses: 1228 num edges: 1483
First 5 poses (file init):
id=1223: x=-7.226028, y=-31.904243, theta=2.038302
id=1224: x=-7.405412, y=-31.351278, theta=1.855394
id=1225: x=-7.584838, y=-30.751843, theta=1.869158
id=1226: x=-7.616041, y=-30.650012, theta=1.852427
id=1227: x=-7.616041, y=-30.650012, theta=1.851765


iteration= 0	 chi2= 5094164.106934	 time= 0.00178789	 cumTime= 0.00178789	 edges= 1483	 schur= 0	 lambda= 8006867.459158	 levenbergIter= 1
iteration= 1	 chi2= 4939391.845232	 time= 0.000630942	 cumTime= 0.00241883	 edges= 1483	 schur= 0	 lambda= 2668955.819719	 levenbergIter= 1
iteration= 2	 chi2= 4568818.318794	 time= 0.000615993	 cumTime= 0.00303483	 edges= 1483	 schur= 0	 lambda= 889651.939906	 levenbergIter= 1
iteration= 3	 chi2= 3919775.040460	 time= 0.000599239	 cumTime= 0.00363407	 edges= 1483	 schur= 0	 lambda= 296550.646635	 levenbergIter= 1
iteration= 4	 chi2= 2722096.327529	 time= 0.000597647	 cumTime= 0.00423171	 edges= 1483	 schur= 0	 lambda= 98850.215545	 levenbergIter= 1
iteration= 5	 chi2= 2080862.025973	 time= 0.000600232	 cumTime= 0.00483194	 edges= 1483	 schur= 0	 lambda= 32950.071848	 levenbergIter= 1
iteration= 6	 chi2= 1798082.421968	 time= 0.000578694	 cumTime= 0.00541064	 edges= 1483	 schur= 0	 lambda= 10983.357283	 levenbergIter= 1
iteration= 7	 chi2= 1610390.6

KeyboardInterrupt: 

In [17]:
from g2o import SparseBlockMatrixX

vertices = graph_slam.optimizer.vertices()
print("len(vertices):", len(vertices))

cov_vertices = []
covariances = SparseBlockMatrixX()
for vertex_idx in vertices:
    v = vertices[vertex_idx]
    if type(v) == g2o.VertexSE2 and not v.fixed():
        cov_vertices.append((v.hessian_index(), v.hessian_index()))
print("Sparse pattern contains", len(cov_vertices), "blocks")
print(cov_vertices)

len(vertices): 6
Sparse pattern contains 2 blocks
[(3, 3), (1, 1)]


In [18]:
print("Computing covariance matrix...")
covariances, ok = graph_slam.optimizer.compute_marginals(cov_vertices)
print("Done, success =", ok)

print("rows:", covariances.rows())
print("cols:", covariances.cols())

Computing covariance matrix...
Done, success = False
rows: 0
cols: 0


In [13]:
print("Covariance matrix:")
for vertex_idx in vertices:
    v = vertices[vertex_idx]
    if type(v) == g2o.VertexSE2 and not v.fixed():
        print(" Working on vertex idx:", vertex_idx)
        print("  Hessian idx is:", v.hessian_index())
        if covariances.rows() > v.hessian_index() and covariances.cols() > v.hessian_index():
            covariance = covariances.block(v.hessian_index(), v.hessian_index());
            print("  Covariance is:", covariance)
        else:
            print("  ERROR: The covariance matrix is not big enough. Something is wrong.")

Covariance matrix:
 Working on vertex idx: 4
  Hessian idx is: 3
  ERROR: The covariance matrix is not big enough. Something is wrong.
 Working on vertex idx: 2
  Hessian idx is: 1
  ERROR: The covariance matrix is not big enough. Something is wrong.
